In [98]:
import torch
import trackpy as tp
import pickle
import matplotlib.pyplot as plt
import numpy as np
from trackpy.utils import validate_tuple
import trackpy.preprocessing as tpp
import torchvision
import scipy.ndimage as nd
import hatzakis_lab_tracking as hlt
device = "cuda"
cpu = lambda x: np.array(np.squeeze(x.detach().cpu()))


In [25]:
with open("args.pickle", "rb") as argfile, open("res.pickle", "rb") as resfile, open("intpic.pickle", "rb") as picfile:
    args = pickle.load(argfile)
    res = pickle.load(resfile)
    pic = pickle.load(picfile)


EOFError: Ran out of input

## Start "bandpass"

In [106]:

# device = "cpu"
# image, lshort, llong, threshold, truncate = args
image = hlt.image_loader_video(r"C:\Users\andre\Documents\tracking_optimizations\emily_tracking\sample_vids\Experiment_Process_001_20220823.tif")
lshort, llong, threshold, truncate = 1, 13, 0, 4
lshort = validate_tuple(lshort, image.ndim)
llong = validate_tuple(llong, image.ndim)
timage = torch.tensor(image.astype(float)).reshape(1, 1, *image.shape).to(device)
padder = (*(np.array(llong) // 2),)*2
padded_timage = torch.nn.functional.pad(timage, padder, "replicate").to(device)

gauss_ker = [tp.masks.gaussian_kernel(sigma, truncate) for sigma in lshort]


In [107]:
padded_timage[:] = 0
timage, padded_timage

(tensor([[[[[2148., 2125., 2094.,  ..., 2113., 2104., 2158.],
            [2139., 2144., 2146.,  ..., 2101., 2089., 2129.],
            [2126., 2171., 2220.,  ..., 2090., 2081., 2107.],
            ...,
            [2155., 2160., 2159.,  ..., 2174., 2128., 2039.],
            [2139., 2161., 2185.,  ..., 2153., 2097., 2044.],
            [2108., 2168., 2243.,  ..., 2123., 2055., 2050.]],
 
           [[2098., 2136., 2207.,  ..., 2068., 2168., 2286.],
            [2118., 2145., 2201.,  ..., 2083., 2135., 2226.],
            [2148., 2162., 2199.,  ..., 2103., 2089., 2139.],
            ...,
            [2075., 2071., 2070.,  ..., 2109., 2141., 2185.],
            [2079., 2075., 2076.,  ..., 2110., 2134., 2156.],
            [2098., 2091., 2090.,  ..., 2105., 2107., 2099.]],
 
           [[2144., 2141., 2143.,  ..., 2291., 2271., 2164.],
            [2140., 2139., 2142.,  ..., 2222., 2204., 2159.],
            [2131., 2140., 2153.,  ..., 2132., 2121., 2168.],
            ...,
            [

In [19]:
# %%timeit
box = tpp.boxcar(image, llong)
filter = torch.full(llong, 1/np.product(llong), dtype = torch.double).reshape(1, 1, *llong).to(device)
meaned = torch.nn.functional.conv2d(padded_timage, filter)


ValueError: Smoothing size must be an odd integer. Round up.

In [77]:
final_coords.is_floating_point()

True

In [75]:
np.issubdtype(final_coords.dtype, torch.float)

TypeError: Cannot interpret 'torch.float64' as a data type

In [78]:
# %%timeit
low = tpp.lowpass(image, lshort, truncate)

lwx = int(truncate * lshort[0] + 0.5)
x = np.arange(-lwx, lwx + 1)
lwy = int(truncate * lshort[1] + 0.5)
y = np.arange(-lwy, lwy + 1)
x = x.reshape(-1, 1)
y = y.reshape(1, -1)
r2 = x**2 + y**2
res = np.exp(r2/(-2*lshort[0]**2))
res /= res.sum()
filter = torch.tensor(res).reshape(1, 1, *res.shape).to(device)

blurred = torch.nn.functional.conv2d(timage, filter, padding = "same")
abs(cpu(blurred) - low).sum()

NameError: name 'lshort' is not defined

In [27]:
result = blurred - meaned
result = torch.where(result >= threshold, result, 0)
(cpu(result) - tpp.bandpass(*args)).__abs__().sum()

NameError: name 'blurred' is not defined

## End bandpass
## Start grey_dilation

In [84]:
with open("grey_args.pckl", "rb") as argfile:
    args = pickle.load(argfile)
image, separation, percentile, margin, precise = args
# percentile = 64
ndim = image.ndim
size = [int(2 * s / np.sqrt(ndim)) for s in separation]
padding = (*(np.array(size) // 2).astype(int),)
timage = torch.tensor(image, dtype = torch.float).reshape(*image.shape)
threshold = tp.find.percentile_threshold(image, percentile)
shape = np.array(image.shape)
threshold

2.0

In [95]:
test = test.to("cuda")

In [88]:
test[:] = 1

In [91]:
test.shape, timage.shape

(torch.Size([1, 1, 512, 512]), torch.Size([512, 512]))

In [4]:
dilation = torch.nn.functional.max_pool2d(timage, size, stride = 1, padding = padding)
old = nd.grey_dilation(image, size, mode = "constant")
(cpu(dilation).astype(int) == old).all()

True

In [5]:
maxima = (image == cpu(dilation)) & (image > threshold)
# maxima = np.nonzero(maxima)

In [6]:
old_pos = np.stack(np.nonzero(maxima)).T

In [112]:
candidates = torch.nonzero((timage == dilation) & (timage > threshold))
pos = candidates[:, 2:]

RuntimeError: The size of tensor a (768) must match the size of tensor b (512) at non-singleton dimension 4

In [8]:
at_boundary = ((pos < torch.tensor(margin)) | (pos > (torch.tensor(shape) - torch.tensor(margin) -1))).any(axis = 1)

In [9]:
candidates[~at_boundary].shape

torch.Size([2031, 4])

In [10]:
(cpu(pos) == old_pos).all()

True

## End dilation
## Start refine_com

In [60]:
with open("com_args.pckl", "rb") as file:
    args = pickle.load(file)
    (raw_image, image, radius, coords, max_iterations,
            shift_thresh, characterize, walkthrough) = args
shift_thresh = 0.5

if not np.issubdtype(coords.dtype, np.integer):
    raise ValueError('The coords array should be of integer datatype')
ndim = image.ndim
isotropic = np.all(radius[1:] == radius[:-1])
mask = tp.masks.binary_mask(radius, ndim).astype(np.uint8)

# Declare arrays that we will fill iteratively through loop.
N = coords.shape[0]
final_coords = np.empty_like(coords, dtype=np.float64)
mass = np.empty(N, dtype=np.float64)
raw_mass = np.empty(N, dtype=np.float64)
if characterize:
    if isotropic:
        Rg = np.empty(N, dtype=np.float64)
    else:
        Rg = np.empty((N, len(radius)), dtype=np.float64)
    ecc = np.empty(N, dtype=np.float64)
    signal = np.empty(N, dtype=np.float64)

ogrid = np.ogrid[[slice(0, i) for i in mask.shape]]  # for center of mass
ogrid = [g.astype(float) for g in ogrid]



In [62]:
# %%timeit -r 7 -n 1
traw = torch.tensor(raw_image).reshape(1, 1, *image.shape).to(device)
timage = torch.tensor(image, dtype = torch.float).reshape(1, 1, *image.shape).to(device)
tradius = torch.tensor(radius).to(device)
gridx = torch.tensor(ogrid[0]).to(device).reshape(1, -1, 1)
gridy = torch.tensor(ogrid[1]).to(device).reshape(1, 1, -1)
tmask = torch.tensor(mask).to(device)
pos = torch.tensor(coords).to(device)
frames = candidates[~at_boundary, 0].reshape(-1, 1, 1)

final_coords = torch.full((pos.shape[0], 2), torch.tensor(float("nan")).to(device), dtype = float).to(device)
final_masses = torch.full((pos.shape[0],), torch.tensor(float("nan")).to(device), dtype = torch.float32).to(device)
final_signals = torch.full((pos.shape[0],), torch.tensor(float("nan")).to(device), dtype = torch.float32).to(device)


not_done = torch.arange(pos.shape[0])
make_inds_for_dim = lambda dim: torch.arange(-radius[dim], radius[dim] + 1)[:, None]

upper_bound = torch.tensor(image.shape).to(device) - 1 - tradius.to(device)
grids = [torch.moveaxis(
    torch.arange(-radius[dim], radius[dim] + 1)[:, None].to(device)
     , 0, dim) 
    for dim in range(image.ndim)]
make_inds_for_dim = lambda dim, pos: grids[dim] + pos[:, dim].reshape(-1, 1, 1)

for i in range(max_iterations):
    neighborhoods = timage[frames, 0, make_inds_for_dim(0, pos), make_inds_for_dim(1, pos)]
    neighborhoods = torch.multiply(neighborhoods, tmask, out = neighborhoods)
    masses = neighborhoods.sum(axis = (1, 2))
    cm_x = (neighborhoods * gridx).sum(axis = (1, 2)) / masses
    cm_y = (neighborhoods * gridy).sum(axis = (1, 2)) / masses
    cm = torch.stack((cm_x, cm_y), axis = 1)
    off_center = cm - torch.tensor(radius).to(device)
    back_shift = off_center < -shift_thresh
    forward_shift = off_center > shift_thresh
    keep_iterating = (back_shift | forward_shift).any(axis = 1)
    if (~keep_iterating).sum() != 0:    
        newly_done = not_done[~keep_iterating]
        
        final_coords[newly_done] = cm[~keep_iterating] - tradius + pos[~keep_iterating]
        final_masses[newly_done] = masses[~keep_iterating]
        final_signals[newly_done] = neighborhoods[~keep_iterating].max()
    

    if keep_iterating.sum() == 0:
        break
    pos[back_shift] -= 1
    pos[forward_shift] += 1
    # # pass
    pos = pos[keep_iterating, :].clip(tradius, upper_bound)
    frames = frames[keep_iterating, :, :]
    not_done = not_done[keep_iterating]

if not keep_iterating.sum() == 0:
    rest = not_done[keep_iterating]
    final_coords[rest] = cm[keep_iterating] - tradius + pos[keep_iterating]
    final_masses[rest] = masses[keep_iterating]
    final_signals[rest] = neighborhoods[keep_iterating].max()

frames = candidates[~at_boundary, 0].reshape(-1, 1, 1)
final_pixels = torch.round(final_coords).to(int)
raw_masses = (traw[frames, 0, make_inds_for_dim(0, final_pixels), make_inds_for_dim(1, final_pixels)] * tmask).sum(axis = (1,2))


In [63]:
with open("outs.pckl", "rb") as file:
    old_raw, old_final_coords, old_signal, old_mass = pickle.load(file)

In [70]:
(cpu(final_coords) == old_final_coords).all()

True

In [68]:
(cpu(final_masses) == old_mass).all()

True

In [66]:
old_raw - cpu(raw_masses)

array([   0.  , 1080.75,    0.  , ...,    0.  ,    0.  ,    0.  ])

In [67]:
np.savetxt("letmesee.csv", old_raw - cpu(raw_masses), fmt = "%s")

In [666]:
for feat, coord in enumerate(coords):
    for iteration in range(max_iterations):
        # Define the circular neighborhood of (x, y).
        rect = tuple([slice(c - r, c + r + 1)
                        for c, r in zip(coord, radius)])
        neighborhood = mask * image[rect]
        cm_n = tp.refine.center_of_mass._safe_center_of_mass(neighborhood, radius, ogrid)
        break
        cm_i = cm_n - radius + coord  # image coords

        off_center = cm_n - radius
        # logger.debug('off_center: %f', off_center)
        if np.all(np.abs(off_center) < shift_thresh):
            break  # Accurate enough.
        # If we're off by more than half a pixel in any direction, move..
        coord[off_center > shift_thresh] += 1
        coord[off_center < -shift_thresh] -= 1
        # Don't move outside the image!
        upper_bound = np.array(image.shape) - 1 - radius
        coord = np.clip(coord, radius, upper_bound).astype(int)
        break
    if feat == 0:
        break

In [3]:
import torch_tracking
import torch
import hatzakis_lab_tracking as hlt
import importlib 
importlib.reload(torch_tracking)
params = hlt.Params(
    lip_int_size = 8,  #originally 15 for first attemps
    lip_BG_size = 70,   # originally 40 for first attemps
    gap_size = 2, # adding gap
    
    dynamic_sep = 7,   # afstand mellem centrum af to partikler, 7 er meget lidt så skal være højere ved lavere densitet
    dynamic_mean_multiplier = 12,  #hvor mange partikler finder den, around 1-3, lavere giver flere partikler
    dynamic_object_size = 11, #diameter used in tp.locate, odd integer
    dynamic_search_range = 4,
    dynamic_memory = 0,
    
    static_sep = 8,
    static_mean_multiplier = 4,
    static_object_size = 11,
)
tvid = torch.tensor(hlt.image_loader_video(r"C:\Users\andre\Documents\tracking_optimizations\emily_tracking\sample_vids\c_20.tif").astype(float), device = "cuda")

In [4]:
t = torch.tensor([1,2,3])

In [19]:
idk = (slice(None),)+(None,)*3

In [21]:
t[idk].shape

torch.Size([3, 1, 1, 1])

In [2]:
torch_tracking.locate(tvid, params.static_object_size, separation = params.static_sep)

RuntimeError: expected stride to be a single integer value or a list of 3 values to match the convolution dimensions, but got stride=[1, 1]